In [2]:
import numpy as np

# Layer 
- 신경망에서 레이어는 하나의 연산이 진행되는 한 노드에서 다음 노드로 변화하는 한 단위를 의미한다.
- 각 레이어는 다음 혹은 이전 레이어와 연결되어 있다. 왼쪽에서 오른쪽으로 기본 진행 방향을 정했을 때, 다음 레이어로 가는 것을 순전파, 이전 레이어로 가는 것을 역전파라고 한다. 

    1) 단순한 계층 (주어진 값에 기본 연산만 수행하는 순/역전파)<br>
    2) ReLU 계층 (활성화함수의 순/역전파)<br>
    3) sigmoid 계층 (활성화함수의 순/역전파)<br>

## 1) Simple Layer

In [7]:
class AddLayer : 
    def __init__(self) : 
        pass
    
    def forward(self, x, y) : 
        return x+y
    
    def backward(self, dout) : 
        dx = dout*1
        dy = dout*1
        return dx, dy

In [2]:
class MulLayer : 
    def __init__(self) : 
        self.x = None
        self.y = None
        
    def forward(self, x, y) : 
        self.x = x 
        self.y = y
        return x * y
    
    def backward(self, dout) : 
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

In [5]:
#forward method check
apple = 100
apple_num = 2
tax = 1.1

#layers
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#foward propagation
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax) 

print(price)

220.00000000000003


In [6]:
#backward method check
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) 

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


### Layer test

In [8]:
#values (data)
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

In [9]:
#layers
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

In [10]:
#forward 
apple_price = mul_apple_layer.forward(apple, apple_num) #(1)
orange_price = mul_orange_layer.forward(orange, orange_num) #(2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price) #(3)
price = mul_tax_layer.forward(all_price, tax) #(4)

In [15]:
#backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice) #(4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) #(3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price) #(2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) #(1)

In [16]:
print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


#### * forward와 backward의 argument와 return값이 서로 스위칭되어있는 것을 알 수 있다.

<br><hr><br>

## 2) ReLU Layer

In [1]:
class ReLU : 
    def __init__(self) : 
        self.mask = None
        
    def forward(self, x) : 
        self.mask = (x<=0) #numpy 배열 x와 같은 shape을 가진 배열을 만들고, 0 이상인 셀은 True, 0 미만인 셀은 False인 배열을 반환
        out = x.copy() 
        out[self.mask] = 0 #numpy 배열인 x 중 0 이하인 값은 0으로 치환
        return out
    
    def backward(self, dout) : 
        dout[self.mask] = 0 #numpy 배열 dout 중 0 이하인 값은 0으로 치환
        dx = dout
        return dx

<br><hr><br>

## 3) sigmoid Layer

In [2]:
class sigmoid : 
    def __init__(self) : 
        self.out = None
        
    def forward(self, x) : 
        out = 1 / (1+np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout) : 
        dx = dout * (1.0 - self.out) * self.out
        return dx

<br><hr><br>

<br><hr><br>

## 4) Affine/Softmax Layer
- 행렬을 내적하면 행렬의 행/열이 달라진다. 기하학에서 행렬의 내적을 affine transformation이라고 한다. <br> eg. matmult( 2\*3, 3\*10) = 2*10

### - 행렬의 내적 예시

In [38]:
X = np.random.rand(2)
W = np.random.rand(2,3)
B = np.random.rand(3)

In [39]:
print(X.shape) #(1,2)
print(W.shape) #(2,3)
print(B.shape) #(1,3)

(2,)
(2, 3)
(3,)


In [40]:
print(X)
print(W)

[ 0.71145685  0.92671537]
[[ 0.3418457   0.64992041  0.66029788]
 [ 0.28904548  0.3595229   0.88325718]]


In [41]:
np.dot(X,W)

array([ 0.51107136,  0.79556572,  1.28830146])

In [42]:
np.matmul(X,W)

array([ 0.51107136,  0.79556572,  1.28830146])

- 이 예시에서 X의 shape은 (1,2)로 2개의 열을 가진 1행의 데이터에 대해 예시를 들었지만, 실제 훈련시에는 (N,M)의 shape을 가진, 즉 M개의 열을 가지는 N개의 행을 사용한다.

###  - 배치용 Affine Layer

In [ ]:
X = np.array([[0,0,0,],
              [10,10,10]])
B = np.array([1,2,3])

In [43]:
class Affine : 
    def __init__(self, W, b) : 
        self.W = W 
        self.b = b 
        self.x = None
        self.dW = None
        self.db = None
        
    
    def forward(self, x) : 
        self.x = x 
        out = np.dot(x, self.W) + self.b
        return out
    
    
    def backward(self, dout) : 
        dx = np.dot(dout, self.W.T) 
        self.dW = np.dot(self.x.T, dout) 
        self.db = np.sum(dout, axis=0)

### - Softmax-with-Loss Layer

In [44]:
#3-5) softmax_function
def softmax_function(x) : 
    return np.exp(x-max(x))/np.sum(np.exp(x-max(x)))

In [45]:
#4-4) CEE (cross_entropy_error, 교차엔트로피오차)
def CEE(y_p, y_t) : #y_p: predicted vector, y_t: actual_vector
    if y.ndim == 1 : 
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    delta = 1e-7
    return -1/y_p.shape[0]*np.sum(y_t*np.log(y_p + delta)) #y_p가 0이 되어 log 계산시 -inf가 나오는 현상 방지

In [46]:
class SofwmaxWithLoss : 
    def __init__(self) : 
        self.loss = None #최종 결과물인 loss값
        self.y = None #predicted Y
        self.t = None #actual Y
        
    
    def forward(self, x, t) : 
        self.t = t
        self.y = softmax_function(x)
        self.loss = CEE(self.y, self.t)
        return self.loss
    
    
    def backward(self, dout=1) : 
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

<br><hr><br>